In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

# Load data
df = pd.read_parquet("transactions_comparison.parquet")

# Feature Engineering
df['time_difference'] = df['swift_time'] - df['xrp_time']
df['fee_difference'] = df['swift_fees'] - df['xrp_fees']

# Prepare features and target
features = ['amount', 'currency', 'source_country', 'destination_country', 'swift_time', 'swift_fees', 'xrp_time', 'xrp_fees']
target = 'xrp_outperforms_swift'

X = df[features]
y = df[target]

# One-hot encoding for categorical features
categorical_features = ['currency', 'source_country', 'destination_country']
numerical_features = ['amount', 'swift_time', 'swift_fees', 'xrp_time', 'xrp_fees']

transformer = ColumnTransformer(transformers=[('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)], remainder='passthrough')

# Model pipeline
model = Pipeline(steps=[('preprocessor', transformer), ('classifier', RandomForestClassifier(random_state=42))])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save the model
joblib.dump(model, 'xrp_outperforms_model.joblib')
print("Model saved as xrp_outperforms_model.joblib")

              precision    recall  f1-score   support

        True       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Model saved as xrp_outperforms_model.joblib
